# 1. Load package

In [1]:
import torch
import torch.utils.data
from torch import optim, nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, accuracy_score, cohen_kappa_score
import pandas as pd


# 2.Load data

In [4]:
# Load dataset
data_path = 'Data_for_MLP/Location_Bin_Num_fetr_sltd.csv'
data = pd.read_csv(data_path)

In [6]:
data

,is_Other_Upper_Limb_Issues,is_Knee_Issues,is_Other_Issues,is_Leg_Issues,is_Foot_Issues,is_Other_Lower_Limb_Issues,is_Ankle_Issues,is_Back_Issues,age_at_injury,height_cm,...,DefensiveStyle,Workload,TwoPointRate,ThreePointRate,ShootingEfficiency,LoadPerGame,EfficiencyPerAge,DefensiveAggressiveness,TurnoverRate,FoulRate
0,1,0,0,0,0,0,0,0,28,218.44,...,2.5,362.7,1.000000,0.000000,0.414,11.7,0.014786,0.9,0.068376,0.205128
1,0,1,0,0,0,0,0,0,24,198.12,...,2.2,1140.0,1.000000,0.000000,0.495,15.2,0.020625,1.1,0.072368,0.138158
2,0,1,0,0,0,0,0,0,29,198.12,...,2.8,328.8,1.000000,0.000000,0.492,27.4,0.016966,0.9,0.065693,0.069343
3,0,0,1,0,0,0,0,0,33,198.12,...,3.9,545.1,1.000000,0.000000,0.426,23.7,0.012909,1.1,0.059072,0.097046
4,0,1,0,0,0,0,0,0,24,195.58,...,3.7,1752.3,0.987654,0.012346,0.462,29.7,0.019250,2.0,0.087542,0.094276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5433,0,1,0,0,0,0,0,0,25,205.74,...,5.2,1416.0,0.692308,0.307692,0.530,23.6,0.021200,2.0,0.033898,0.114407
5434,0,0,0,0,0,0,1,0,30,190.50,...,2.9,1392.3,0.804054,0.195946,0.518,27.3,0.017267,0.8,0.058608,0.040293
5435,0,0,0,0,0,0,1,0,29,195.58,...,5.2,1641.5,0.563107,0.436893,0.472,24.5,0.016276,1.9,0.073469,0.102041
5436,0,0,0,0,0,0,1,0,29,198.12,...,2.6,844.6,0.714286,0.285714,0.397,20.6,0.013690,0.7,0.067961,0.082524


# 3. Prepare variables

In [320]:
# Prepare features and target for a single binary variable
target_column = 'is_Other_Issues'
features = data[[col for col in data.columns if not col.startswith('is_')]]
target = data[target_column]

# Normalize features
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.4)

# Convert to PyTorch tensors
train_data = torch.tensor(X_train, dtype=torch.float32)
train_target = torch.tensor(y_train.values, dtype=torch.long)
test_data = torch.tensor(X_test, dtype=torch.float32)
test_target = torch.tensor(y_test.values, dtype=torch.long)

# 4. Modeling

In [323]:
# Create DataLoader for training and testing
batch_size = 200
train_loader = torch.utils.data.DataLoader(
    dataset=torch.utils.data.TensorDataset(train_data, train_target),
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=torch.utils.data.TensorDataset(test_data, test_target),
    batch_size=batch_size,
    shuffle=False
)


In [325]:

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 2),  # Binary classification
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.model(x)

# Initialize model
input_size = features.shape[1]
learning_rate = 0.0002
epochs = 500
net = MLP(input_size)
optimizer = optim.Adam(net.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()


In [327]:
# DataFrame to store results
results_df = pd.DataFrame(columns=['MLP_Accuracy', 'MLP_Precision', 'MLP_Recall', 'MLP_Kappa'], index=[target_column])

# Training loop
for epoch in range(epochs):
    net.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        logits = net(data)
        loss = criterion(logits, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(f"Train Epoch: {epoch + 1} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}")

    # Evaluate on the test set
    net.eval()
    test_loss = 0
    correct = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for data, target in test_loader:
            logits = net(data)
            test_loss += criterion(logits, target).item()
            pred = logits.argmax(dim=1)
            correct += pred.eq(target).sum().item()

            all_preds.extend(pred.numpy())
            all_targets.extend(target.numpy())

    test_loss /= len(test_loader.dataset)
    accuracy = accuracy_score(all_targets, all_preds)
    precision = precision_score(all_targets, all_preds)
    recall = recall_score(all_targets, all_preds)
    kappa = cohen_kappa_score(all_targets, all_preds)

    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Kappa: {kappa:.4f}\n")


Train Epoch: 1 [0/3262] Loss: 0.672417
Train Epoch: 1 [2000/3262] Loss: 0.556167

Test set: Average loss: 0.0025, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 2 [0/3262] Loss: 0.511701
Train Epoch: 2 [2000/3262] Loss: 0.443567

Test set: Average loss: 0.0021, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 3 [0/3262] Loss: 0.407899
Train Epoch: 3 [2000/3262] Loss: 0.382058

Test set: Average loss: 0.0019, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 4 [0/3262] Loss: 0.384962
Train Epoch: 4 [2000/3262] Loss: 0.350806

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 5 [0/3262] Loss: 0.364632
Train Epoch: 5 [2000/3262] Loss: 0.358243

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 6 [0/3262] Loss: 0.355914
Train Epoch: 6 [2000/3262] Loss: 0.335369

Test set

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_


Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 8 [0/3262] Loss: 0.362527
Train Epoch: 8 [2000/3262] Loss: 0.351690

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 9 [0/3262] Loss: 0.376049
Train Epoch: 9 [2000/3262] Loss: 0.361389

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 10 [0/3262] Loss: 0.340608
Train Epoch: 10 [2000/3262] Loss: 0.355062

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 11 [0/3262] Loss: 0.365263
Train Epoch: 11 [2000/3262] Loss: 0.355700

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 12 [0/3262] Loss: 0.354352
Train Epoch: 12 [2000/3262] Loss: 0.334996

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_


Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 18 [0/3262] Loss: 0.329208
Train Epoch: 18 [2000/3262] Loss: 0.373937

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 19 [0/3262] Loss: 0.358739
Train Epoch: 19 [2000/3262] Loss: 0.354036

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 20 [0/3262] Loss: 0.343823
Train Epoch: 20 [2000/3262] Loss: 0.368752

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 21 [0/3262] Loss: 0.373746
Train Epoch: 21 [2000/3262] Loss: 0.348620

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 22 [0/3262] Loss: 0.353858
Train Epoch: 22 [2000/3262] Loss: 0.363873

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.

...

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_

Train Epoch: 475 [0/3262] Loss: 0.353262
Train Epoch: 475 [2000/3262] Loss: 0.353262

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 476 [0/3262] Loss: 0.373262
Train Epoch: 476 [2000/3262] Loss: 0.343262

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 477 [0/3262] Loss: 0.353262
Train Epoch: 477 [2000/3262] Loss: 0.358262

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 478 [0/3262] Loss: 0.348262
Train Epoch: 478 [2000/3262] Loss: 0.323262

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 479 [0/3262] Loss: 0.363261
Train Epoch: 479 [2000/3262] Loss: 0.368262

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 480 [0/3262] Loss: 0.343262
Train Epoch: 480 [2000/3262] 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_

Train Epoch: 483 [2000/3262] Loss: 0.368262

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 484 [0/3262] Loss: 0.338262
Train Epoch: 484 [2000/3262] Loss: 0.328262

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 485 [0/3262] Loss: 0.343262
Train Epoch: 485 [2000/3262] Loss: 0.353262

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 486 [0/3262] Loss: 0.333262
Train Epoch: 486 [2000/3262] Loss: 0.328262

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 487 [0/3262] Loss: 0.383262
Train Epoch: 487 [2000/3262] Loss: 0.318262

Test set: Average loss: 0.0018, Accuracy: 0.9669, Precision: 0.0000, Recall: 0.0000, Kappa: 0.0000

Train Epoch: 488 [0/3262] Loss: 0.358262
Train Epoch: 488 [2000/3262] Loss: 0.358262

Test set: Average loss: 0

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_

In [328]:
# Store results in DataFrame
results_df.loc[target_column] = [accuracy, precision, recall, kappa]

# Print the results DataFrame
print("Final Results:")
print(results_df)

Final Results:
                MLP_Accuracy MLP_Precision MLP_Recall MLP_Kappa
is_Other_Issues     0.966912           0.0        0.0       0.0


In [331]:
MLP_results = pd.concat([MLP_results, results_df], ignore_index=False)

In [333]:
MLP_results

,MLP_Accuracy,MLP_Precision,MLP_Recall,MLP_Kappa
is_Knee_Issues,0.746324,0.505556,0.16426,0.14065
is_Ankle_Issues,0.817096,0.384615,0.02551,0.026019
is_Foot_Issues,0.903952,0.0,0.0,0.0
is_Leg_Issues,0.901654,0.0,0.0,0.0
is_Back_Issues,0.888327,0.0,0.0,0.0
is_Other_Upper_Limb_Issues,0.871324,0.0,0.0,0.0
is_Other_Lower_Limb_Issues,0.914522,0.0,0.0,0.0
is_Other_Issues,0.966912,0.0,0.0,0.0


In [335]:
MLP_results_location_ftr_sltd = MLP_results

In [337]:
MLP_results_location_ftr_sltd

,MLP_Accuracy,MLP_Precision,MLP_Recall,MLP_Kappa
is_Knee_Issues,0.746324,0.505556,0.16426,0.14065
is_Ankle_Issues,0.817096,0.384615,0.02551,0.026019
is_Foot_Issues,0.903952,0.0,0.0,0.0
is_Leg_Issues,0.901654,0.0,0.0,0.0
is_Back_Issues,0.888327,0.0,0.0,0.0
is_Other_Upper_Limb_Issues,0.871324,0.0,0.0,0.0
is_Other_Lower_Limb_Issues,0.914522,0.0,0.0,0.0
is_Other_Issues,0.966912,0.0,0.0,0.0


In [339]:
MLP_results_location_ftr_sltd.to_csv('MLP_results_location_ftr_sltd.csv', index=True) 